# [ドライバ](https://www.tensorflow.org/agents/tutorials/4_drivers_tutorial)

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf


from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.policies import random_py_policy
from tf_agents.policies import random_tf_policy
from tf_agents.metrics import py_metrics
from tf_agents.metrics import tf_metrics
from tf_agents.drivers import py_driver
from tf_agents.drivers import dynamic_episode_driver

## Pythonドライバ

`PyDriver`クラスは、Python環境、Pythonポリシー、および各ステップで更新するオブザーバーのリストを受け取ります。主なメソッドは`run()`で、終了基準（ステップ数が`max_steps` に達するか、エピソード数が`max_episodes`に達する）が少なくとも1つ満たされるまで、ポリシーに基づいた行動を環境で実行します。

CartPole環境での実行：

In [2]:
env = suite_gym.load('CartPole-v0')
policy = random_py_policy.RandomPyPolicy(time_step_spec=env.time_step_spec(), 
                                         action_spec=env.action_spec())
replay_buffer = []
metric = py_metrics.AverageReturnMetric()
observers = [replay_buffer.append, metric]
driver = py_driver.PyDriver(
    env, policy, observers, max_steps=20, max_episodes=1)

In [4]:
initial_time_step = env.reset()
final_time_step, _ = driver.run(initial_time_step)

print('Replay Buffer:')
for traj in replay_buffer:
    print(traj)

print('Average Return: ', metric.result())

Replay Buffer:
Trajectory(step_type=array(0, dtype=int32), observation=array([ 0.03291266, -0.02432417,  0.00065954,  0.02049842], dtype=float32), action=array(1), policy_info=(), next_step_type=array(1, dtype=int32), reward=array(1., dtype=float32), discount=array(1., dtype=float32))
Trajectory(step_type=array(1, dtype=int32), observation=array([ 0.03242618,  0.17078832,  0.00106951, -0.27197635], dtype=float32), action=array(0), policy_info=(), next_step_type=array(1, dtype=int32), reward=array(1., dtype=float32), discount=array(1., dtype=float32))
Trajectory(step_type=array(1, dtype=int32), observation=array([ 0.03584194, -0.02434888, -0.00437002,  0.02104372], dtype=float32), action=array(1), policy_info=(), next_step_type=array(1, dtype=int32), reward=array(1., dtype=float32), discount=array(1., dtype=float32))
Trajectory(step_type=array(1, dtype=int32), observation=array([ 0.03535496,  0.17083547, -0.00394915, -0.27301478], dtype=float32), action=array(0), policy_info=(), next_st

## TensorFlowドライバ

また、TensorFlowには、Pythonドライバと機能的に似ているドライバがありますが、TensorFlowドライバでは、TF環境、TFポリシー、TFオブザーバーなどを使用します。現在、次の2つのTensorFlowドライバがあります。

- `DynamicStepDriver`：指定された数の（有効な）環境ステップの後に終了します。
- `DynamicEpisodeDriver`：指定された数のエピソードの後に終了します。

実際の`DynamicEpisodeDriver`の例を見てみましょう。

In [5]:
env = suite_gym.load('CartPole-v0')
tf_env = tf_py_environment.TFPyEnvironment(env)

tf_policy = random_tf_policy.RandomTFPolicy(action_spec=tf_env.action_spec(),
                                            time_step_spec=tf_env.time_step_spec())

In [6]:
num_episodes = tf_metrics.NumberOfEpisodes()
env_steps = tf_metrics.EnvironmentSteps()
observers = [num_episodes, env_steps]
driver = dynamic_episode_driver.DynamicEpisodeDriver(
    tf_env, tf_policy, observers, num_episodes=2)

# Initial driver.run will reset the environment and initialize the policy.
final_time_step, policy_state = driver.run()

print('final_time_step', final_time_step)
print('Number of Steps: ', env_steps.result().numpy())
print('Number of Episodes: ', num_episodes.result().numpy())

final_time_step TimeStep(step_type=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>, reward=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>, discount=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>, observation=<tf.Tensor: shape=(1, 4), dtype=float32, numpy=
array([[-0.00442522, -0.04254914, -0.01799237,  0.03740452]],
      dtype=float32)>)
Number of Steps:  39
Number of Episodes:  2


In [7]:
# Continue running from previous state
final_time_step, _ = driver.run(final_time_step, policy_state)

print('final_time_step', final_time_step)
print('Number of Steps: ', env_steps.result().numpy())
print('Number of Episodes: ', num_episodes.result().numpy())

final_time_step TimeStep(step_type=<tf.Tensor: shape=(1,), dtype=int32, numpy=array([0], dtype=int32)>, reward=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.], dtype=float32)>, discount=<tf.Tensor: shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>, observation=<tf.Tensor: shape=(1, 4), dtype=float32, numpy=
array([[-0.03343501, -0.0185016 ,  0.0439446 , -0.04853262]],
      dtype=float32)>)
Number of Steps:  67
Number of Episodes:  4
